# Generic system-near OpenMath serialization for Python: a demo

## A first OpenMath serialization and deserialization: Sage integers

In [1]:
from openmath.convert import to_python
from openmath_pickle import to_openmath, test_openmath

A Sage integer is serialized as an application of the function `sage.rings.integer.make_integer` applied on a (base 32?) string representation:

In [2]:
to_openmath(42)

OMApplication(elem=OMSymbol(name='make_integer', cd='sage.rings.integer', id=None, cdbase='http://python.org'), arguments=[OMBytes(bytes='1a', id=None)], id=None, cdbase=None)

For deserialization, we can use the plain openmath deserialization from [py-openmath](https://github.com/OpenMath/py-openmath):

In [3]:
to_python(_)

42

This really is a Sage integer:

In [4]:
type(_)

<type 'sage.rings.integer.Integer'>

Note that the above is a **system-near OpenMath serialization**. Deserializing to another system will further require **alignments** with its own constructors (a Rosetta stone). This will be the job of the Math-in-the-Middle engine.

This utility checks that serialization and deserialization gives an equal object of the same type:

In [5]:
test_openmath(1)

## Serialization of basic Python types

For basic Python types, we get the standard OpenMath serialization:

In [6]:
to_openmath(42r)

OMInteger(integer=42, id=None)

In [7]:
to_openmath('coucou')

OMBytes(bytes='coucou', id=None)

In [8]:
to_openmath(None)

OMSymbol(name='None', cd='__builtin__', id=None, cdbase='http://python.org')

In [9]:
l = [1r,2r,3r]
to_openmath(l)

OMApplication(elem=OMSymbol(name='list', cd='list1', id=None, cdbase='http://www.openmath.org/cd'), arguments=[OMInteger(integer=1, id=None), OMInteger(integer=2, id=None), OMInteger(integer=3, id=None)], id=None, cdbase=None)

In [10]:
test_openmath(l)

In [11]:
d = {1r: 'foo', 'bar': 2r}
to_openmath(d)

OMApplication(elem=OMSymbol(name='dict', cd='__builtin__', id=None, cdbase='http://python.org'), arguments=[OMApplication(elem=OMSymbol(name='list', cd='list1', id=None, cdbase='http://www.openmath.org/cd'), arguments=[OMApplication(elem=OMSymbol(name='list', cd='list1', id=None, cdbase='http://www.openmath.org/cd'), arguments=[OMInteger(integer=1, id=None), OMBytes(bytes='foo', id=None)], id=None, cdbase=None), OMApplication(elem=OMSymbol(name='list', cd='list1', id=None, cdbase='http://www.openmath.org/cd'), arguments=[OMBytes(bytes='bar', id=None), OMInteger(integer=2, id=None)], id=None, cdbase=None)], id=None, cdbase=None)], id=None, cdbase=None)

In [12]:
for obj in [None, int(42), 'coucou', l, d]:
    test_openmath(obj)

## Non trivial examples: Permutations groups and elements thereof

In [13]:
G = DihedralGroup(3)

In [14]:
to_openmath(G)

OMApplication(elem=OMSymbol(name='unreduce', cd='sage.structure.unique_representation', id=None, cdbase='http://python.org'), arguments=[OMSymbol(name='DihedralGroup', cd='sage.groups.perm_gps.permgroup_named', id=None, cdbase='http://python.org'), OMApplication(elem=OMSymbol(name='tuple', cd='__builtin__', id=None, cdbase='http://python.org'), arguments=[OMApplication(elem=OMSymbol(name='list', cd='list1', id=None, cdbase='http://www.openmath.org/cd'), arguments=[OMApplication(elem=OMSymbol(name='make_integer', cd='sage.rings.integer', id=None, cdbase='http://python.org'), arguments=[OMBytes(bytes='3', id=None)], id=None, cdbase=None)], id=None, cdbase=None)], id=None, cdbase=None), OMApplication(elem=OMSymbol(name='dict', cd='__builtin__', id=None, cdbase='http://python.org'), arguments=[OMApplication(elem=OMSymbol(name='list', cd='list1', id=None, cdbase='http://www.openmath.org/cd'), arguments=[], id=None, cdbase=None)], id=None, cdbase=None)], id=None, cdbase=None)

In [15]:
to_python(_)

Dihedral group of order 6 as a permutation group

In [17]:
g = G.an_element(); g

(1,2,3)

In [18]:
to_openmath(g)

OMApplication(elem=OMSymbol(name='make_permgroup_element_v2', cd='sage.groups.perm_gps.permgroup_element', id=None, cdbase='http://python.org'), arguments=[OMApplication(elem=OMSymbol(name='unreduce', cd='sage.structure.unique_representation', id=None, cdbase='http://python.org'), arguments=[OMSymbol(name='DihedralGroup', cd='sage.groups.perm_gps.permgroup_named', id=None, cdbase='http://python.org'), OMApplication(elem=OMSymbol(name='tuple', cd='__builtin__', id=None, cdbase='http://python.org'), arguments=[OMApplication(elem=OMSymbol(name='list', cd='list1', id=None, cdbase='http://www.openmath.org/cd'), arguments=[OMApplication(elem=OMSymbol(name='make_integer', cd='sage.rings.integer', id=None, cdbase='http://python.org'), arguments=[OMBytes(bytes='3', id=None)], id=None, cdbase=None)], id=None, cdbase=None)], id=None, cdbase=None), OMApplication(elem=OMSymbol(name='dict', cd='__builtin__', id=None, cdbase='http://python.org'), arguments=[OMApplication(elem=OMSymbol(name='list', cd

In [19]:
to_python(_)

(1,2,3)